# Median Runner

This notebook demonstrates how to create a grading workflow using PyBryt.

In [10]:
import pybryt

This demo has the following directory structure. This notebook, `index.ipynb`, runs PyBryt, `median.ipynb` is the assignment reference implementation, and `submissions` contains notebooks with student code in them.

In [11]:
%%bash
tree

.
├── index.ipynb
├── median-ref.pkl
├── median.ipynb
└── submissions
    ├── subm01.ipynb
    ├── subm02.ipynb
    ├── subm03.ipynb
    ├── subm04.ipynb
    ├── subm05.ipynb
    └── subm06.ipynb

1 directory, 9 files


## Reference Implementations

If you have marked up a reference implementation, like the one in [`median.ipynb`](median.ipynb), you can load this reference using `pybryt.ReferenceImplementation.compile`. Because references are relatively static and can take some time to execute, you can pickle the reference implementations to a file with `pybryt.ReferenceImplementation.dump`.

In [12]:
ref = pybryt.ReferenceImplementation.compile("median.ipynb")
ref.dump("median-ref.pkl")

To load a pickled reference implementation, use `pybryt.ReferenceImplementation.load`:

In [13]:
ref = pybryt.ReferenceImplementation.load("median-ref.pkl")
ref

## Assessing Submissions

To use PyBryt for grading multiple submissions, you can build a reproducible grading pipeline for an arbitrary number of submissions. To grab the submission notebook paths, the cell below uses `glob.glob`.

In [14]:
from glob import glob
subms = sorted(glob("submissions/*.ipynb"))
subms

['submissions/subm01.ipynb',
 'submissions/subm02.ipynb',
 'submissions/subm03.ipynb',
 'submissions/subm04.ipynb',
 'submissions/subm05.ipynb',
 'submissions/subm06.ipynb']

To use PyBryt to grade a student's submission, a `pybryt.StudentImplementation` must be created from that submission. The constructor takes the path to the notebook as its only positional argument. It is in this step that the student's code is executed, so this cell will need to be rerun whenever changes are made to the submission notebooks.

In [15]:
student_impls = []
for subm in subms:
    student_impls.append(pybryt.StudentImplementation(subm))

student_impls

Once you have created the `pybryt.StudentImplementation` objects, use the `pybryt.StudentImplementation.check` method to run the check of a submission against a reference implementation. This method returns a single `pybryt.ReferenceResult` or a list of them, depending on the argument passed to `check`. In the cell below, the results are collected into a list.

In [16]:
results = []
for si in student_impls:
    results.append(si.check(ref))

results

[ReferenceResult([
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value)
 ]),
 ReferenceResult([
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value)
 ]),
 ReferenceResult([
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value),
   AnnotationResult(satisfied=False, annotation=pybryt.Value),
   AnnotationResult(satisfied=True, annotation=pybryt.Value)


To view the results in a concise manner, the `pybryt.ReferenceResult` class has some helpful instance variables. You can also get information about the memory footprint, such as the number of steps, from the `pybryt.StudentImplementation` class.

In [17]:
from textwrap import indent
for sp, si, res in zip(subms, student_impls, results):
    print(f"SUBMISSION: {sp}")
    print(f"  EXECUTION STEPS: {si.steps}") # the number of steps in the execution

    # res.messages is a list of messages returned by the reference during grading
    messages = "\n".join(res.messages) 
    
    # res.correct is a boolean for whether the reference was satisfied
    message = f"SATISFIED: {res.correct}\nMESSAGES:\n{indent(messages, '  - ')}"
    
    # some pretty-printing
    print(indent(message, "  "))
    print("\n")

SUBMISSION: submissions/subm01.ipynb
  EXECUTION STEPS: 106842
  SATISFIED: True
  MESSAGES:
    - SUCCESS: Sorted the sample correctly
    - SUCCESS: Computed the size of the sample
    - SUCCESS: Sorted the sample correctly
    - SUCCESS: Computed the size of the sample
    - computed the correct median


SUBMISSION: submissions/subm02.ipynb
  EXECUTION STEPS: 101316
  SATISFIED: False
  MESSAGES:
    - ERROR: The sample was not sorted
    - ERROR: Did not capture the size of the set to determine if it is odd or even
    - ERROR: The sample was not sorted
    - ERROR: Did not capture the size of the set to determine if it is odd or even
    - computed the correct median


SUBMISSION: submissions/subm03.ipynb
  EXECUTION STEPS: 102576
  SATISFIED: False
  MESSAGES:
    - SUCCESS: Sorted the sample correctly
    - ERROR: Did not capture the size of the set to determine if it is odd or even
    - SUCCESS: Sorted the sample correctly
    - ERROR: Did not capture the size of the set to de

You can also turn the reference result objects into a JSON-friendly dictionary format for further processing:

In [18]:
res = results[0]
res.to_dict()

{'group': None,
 'results': [{'satisfied': True,
   'satisfied_at': 48156,
   'annotation': {'name': 'Annotation 1',
    'group': None,
    'limit': None,
    'success_message': 'SUCCESS: Sorted the sample correctly',
    'failure_message': 'ERROR: The sample was not sorted',
    'children': [],
    'type': 'value',
    'invariants': [],
    'tol': 0},
   'children': []},
  {'satisfied': True,
   'satisfied_at': 48151,
   'annotation': {'name': 'Annotation 2',
    'group': None,
    'limit': None,
    'success_message': 'SUCCESS: Computed the size of the sample',
    'failure_message': 'ERROR: Did not capture the size of the set to determine if it is odd or even',
    'children': [],
    'type': 'value',
    'invariants': [],
    'tol': 0},
   'children': []},
  {'satisfied': True,
   'satisfied_at': 48156,
   'annotation': {'name': 'Annotation 3',
    'group': None,
    'limit': None,
    'success_message': 'SUCCESS: Sorted the sample correctly',
    'failure_message': 'ERROR: The sam